In [8]:
from bs4 import BeautifulSoup
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np
import re
import requests
from lxml import html

Start with webscraping all of the New York Time recipe pages that are referenced in the Kaggle Dataset. The data that we are interested in is the recipe ratings and the number of ingredients that each recipe requires.

In [9]:
data=pd.read_csv('nyt_recipe_comments_Jun25.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

/Users/Eva/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
data.head()

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,userURL,picURL,commentTitle,...,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,recipe_id,recipe_name
0,20158342.0,approved,20158342.0,1570140.0,Christa,<br/>,NaN,NaN,NaN,<br\//>,...,NaN,NaN,1.0,comment,0.0,0.0,20158342.0,False,1015646,shredded oxtail salad with mustard and shallot
1,110917145.0,approved,110917145.0,87444971.0,Lindsey,NaN,NaN,NaN,NaN,<br\//>,...,NaN,NaN,1.0,comment,0.0,0.0,110917145.0,False,1012940,aunt gens biscotti
2,104240855.0,approved,104240855.0,68602151.0,VFR Marc,NaN,NaN,NaN,NaN,<br\//>,...,NaN,NaN,1.0,comment,0.0,0.0,104240855.0,False,1012940,aunt gens biscotti
3,16837960.0,approved,16837960.0,30298351.0,Joanne,<br/>,NaN,NaN,NaN,<br\//>,...,NaN,NaN,1.0,comment,0.0,0.0,16837960.0,False,1017810,turkey dinner canapes
4,111016114.0,approved,111016114.0,57140404.0,Rachel,NaN,NaN,NaN,NaN,<br\//>,...,NaN,NaN,1.0,comment,0.0,0.0,111016114.0,False,1019207,lalla mussa dal


In [11]:
def create_urls(end, urls):
    base_url='https://cooking.nytimes.com/recipes/'+end
    urls.append(base_url)

In [56]:
end_url=[]

group_df=data.groupby(['recipe_id','recipe_name']).sum()
group_df=pd.DataFrame(group_df)
group_df=group_df.reset_index()


id_dict={}

for row in range(len(group_df)):
    recipe_id=str(group_df.loc[row,'recipe_id'])
    id_dict[recipe_id]=[]
    recipe_name=group_df.loc[row,'recipe_name']
    recipe_name=recipe_name.replace(" ","-")
    end_url.append(recipe_id+'-'+recipe_name)
    
urls=[]    
for end in end_url:
    create_urls(end, urls)

In [57]:
len(id_dict)

8436

In [61]:
def scrape_recipe_page(main_url):
    soup = BeautifulSoup(requests.get(main_url).text, "html.parser")
    
    #Scrape recipe rating
    try:
        rating_text=soup.find("script", type="application/ld+json").text
        begin_pos=rating_text.find("ratingValue")
        rating_text=rating_text[begin_pos:]
        end_pos=rating_text.find(":")
        rating=rating_text[end_pos+1:end_pos+2]
    except:
        rating=-1

    #Scrape the number of ingredients
    num_ingredients=len(soup.find_all("span", class_="ingredient-name"))
    
    #Add back to dict to later join with dataframe
    pos_num=re.search(r"\d", main_url)
    if pos_num:

        end_num=main_url[pos_num.start():].find('-')
        #try:
        #print(main_url[pos_num.start():pos_num.start()+end_num])
        if len(id_dict[main_url[pos_num.start():pos_num.start()+end_num]])<2:
            id_dict[main_url[pos_num.start():pos_num.start()+end_num]].extend([num_ingredients,rating])
        else:
            pass
        #except:
            #print(main_url[pos_num.start():end_num])
            
            
            
        #id_dict[main_url[pos_num.start():pos_num.start()+7]]=num_ingredients
    #id_dict[main_url[re.search(r"\d", main_url):main_url[re.search(r"\d", main_url):].find('-')]]=num_ingredients
    

In [ ]:
for url in urls:
    scrape_recipe_page(url)

In [60]:
id_dict

{'852': [10, '4'],
 '1466': [14, '4'],
 '1467': [10, '4'],
 '1468': [12, '4'],
 '1469': [6, '4'],
 '1470': [7, '5'],
 '1473': [7, '4'],
 '1475': [6, '4'],
 '1746': [6, '4'],
 '1801': [6, '4'],
 '3783': [8, '5'],
 '4406': [13, '5'],
 '4599': [6, '5'],
 '5294': [7, '4'],
 '5612': [11, '4'],
 '5613': [10, '5'],
 '6970': [6, '4'],
 '8665': [8, '5'],
 '9558': [12, '5'],
 '11206': [13, '4'],
 '11376': [4, '5'],
 '11404': [15, '4'],
 '12141': [4, '5'],
 '12142': [15, '5'],
 '12144': [8, '4'],
 '12145': [17, '5'],
 '12146': [9, '5'],
 '12148': [11, '5'],
 '12151': [8, '4'],
 '12152': [6, '5'],
 '12153': [11, '3'],
 '12154': [10, '4'],
 '12155': [10, '4'],
 '12156': [8, '4'],
 '12157': [7, '4'],
 '12158': [19, '5'],
 '12159': [19, '5'],
 '12160': [2, '3'],
 '12166': [7, '4'],
 '12168': [9, '5'],
 '12169': [18, '5'],
 '12170': [13, '4'],
 '12173': [17, '4'],
 '12176': [9, '4'],
 '12177': [14, '5'],
 '12178': [12, '4'],
 '12179': [12, '4'],
 '12180': [9, '5'],
 '12181': [10, '4'],
 '12182': [9, '

In [ ]:
ingredient_rating_df=pd.DataFrame.from_dict(id_dict, orient='index',
                       columns=['num_ingredients', 'rating'])

ingredient_rating_df.index.name='recipe_id'      
ingredient_rating_df

In [ ]:
ingredient_rating_df.to_csv('ingredient_rating_df.csv')

In [ ]:
ratings=pd.read_csv('ingredient_rating_df.csv')
ratings

In [ ]:
#recipes that do not have ratings are represented with a -1, replace

In [25]:
len(np.unique(data['recipe_id']))

8436

In [28]:
len(ratings['recipe_id'])

173

In [26]:
data.join(ratings.set_index('recipe_id'), on='recipe_id', how='left')

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,userURL,picURL,commentTitle,...,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,recipe_id,recipe_name,num_ingredients,rating
0,20158342.0,approved,20158342.0,1570140.0,Christa,<br/>,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,20158342.0,False,1015646,shredded oxtail salad with mustard and shallot,9.0,3
1,110917145.0,approved,110917145.0,87444971.0,Lindsey,NaN,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,110917145.0,False,1012940,aunt gens biscotti,11.0,4
2,104240855.0,approved,104240855.0,68602151.0,VFR Marc,NaN,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,104240855.0,False,1012940,aunt gens biscotti,11.0,4
3,16837960.0,approved,16837960.0,30298351.0,Joanne,<br/>,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,16837960.0,False,1017810,turkey dinner canapes,6.0,5
4,111016114.0,approved,111016114.0,57140404.0,Rachel,NaN,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,111016114.0,False,1019207,lalla mussa dal,13.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389958,100972892.0,approved,100972892.0,66742848.0,ChiaviDiBasso,NaN,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,100972892.0,False,1016816,sweet tart dough,NaN,NaN
389959,27563025.0,approved,27563025.0,76240603.0,ysenta,<br/>,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,27563025.0,False,1016816,sweet tart dough,NaN,NaN
389960,18888540.0,approved,18888540.0,36330041.0,Kristin yates,<br/>,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,18888540.0,False,1016816,sweet tart dough,NaN,NaN
389961,17391420.0,approved,17391420.0,14121543.0,rprp2,<br/>,NaN,NaN,NaN,<br\//>,...,1.0,comment,0.0,0.0,17391420.0,False,1016816,sweet tart dough,NaN,NaN


Now we are going to use the dataset to try and develop a model that can predict a recipe's rating.
Some things that we will look at are:
- number of ingredients that a recipe requires
- the average sentiment of the comments left per recipe (can add a column to our dataset that has the average sentiment value of the comments left on a 
- the number of replies? 
- can also create a model and then run feature importance
- do we want to add calories and other nutritional information to the dataframe?
- regression model 
    - random forest regression 
- exploratory data analysis:
    - distributions of comments (how often are there no or many comments), sentiment analysis, ratings (usually bimodal)
    - word clouds